In [ ]:
import json
import numpy as np
from imageio import imread
import os
import random
import matplotlib.pyplot as plt
from matplotlib import patches,  lines
from matplotlib.patches import Polygon
import colorsys
from IPython.core.display import display, HTML

ORIGINAL_IMAGES_DIR = './dataset/original_images'
PADDED_IMAGES_DIR = './dataset/padded_images'
# PREDICTIONS_JSON_PATH = './dataset/mask_rcnn/results/vein/predictions_20180824T114300/predictions.json'
PREDICTIONS_JSON_PATH = './dataset/mask_rcnn/results/vein/predictions_20180827T092412/predictions.json'
LABELBOX_JSON_PATH = './dataset/labelbox_parser/input/labelbox.json'

LABELBOX_PROJECT_URL = 'https://app.labelbox.com/projects/cjklwemc4qm430732gly212ti'
LABELBOX_CLASS_NAMES = ['Sulphide/Partial Sulphide', 'Pure Quartz Carbonate']


In [ ]:
def random_colors(N, bright=True):
    brightness = 1.0 if bright else 0.7
    hsv = [(i / N, 1, brightness) for i in range(N)]
    colors = list(map(lambda c: colorsys.hsv_to_rgb(*c), hsv))
    random.shuffle(colors)
    return colors


In [ ]:
def display_instances(title, image, instances, show_label=True):
    colors = random_colors(len(instances))
    height, width = image.shape[:2]
    
    _, ax = plt.subplots(figsize=(18, 18))
    ax.set_title(title)
    ax.set_xticks(range(0, width, 400))
    
    for i, instance in enumerate(instances):
        color = colors[i]
        for verts in instance['polygons']:
            p = Polygon(verts, facecolor="none", edgecolor=color)
            ax.add_patch(p)
            
        if show_label:
            x1, y1, x2, y2 = instance['bounding_box']
            p = patches.Rectangle((x1, y1), x2 - x1, y2 - y1, linewidth=2,
                                  alpha=0.7, linestyle="dashed",
                                  edgecolor=color, facecolor='none')
            ax.add_patch(p)

            caption = instance['class'][0]
            if 'score' in instance.keys():
                caption += " {:.3f}".format(instance['score'])
                
            ax.text(x1, y1 - 10, caption, color=color, size=11, backgroundcolor="none")

    ax.imshow(image)
    plt.show()


In [ ]:
class BoundingBox(object):
    """
    A 2D bounding box
    """
    def __init__(self, points):
        if len(points) == 0:
            raise ValueError("Can't compute bounding box of empty list")

        self.minx, self.miny = float("inf"), float("inf")
        self.maxx, self.maxy = float("-inf"), float("-inf")
        
        for x, y in points:
            # Set min coords
            if x < self.minx:
                self.minx = x
            if y < self.miny:
                self.miny = y

            # Set max coords
            if x > self.maxx:
                self.maxx = x
            elif y > self.maxy:
                self.maxy = y


In [ ]:
def parse_labelbox(json_path):
    labelbox_json = json.load(open(json_path))

    labelbox_images = {}
    for item in labelbox_json:
        labels = item['Label']
        if labels == 'Skip':
            continue

        image_name = '{}.png'.format(os.path.splitext(item['External ID'])[0])
        image = imread(os.path.join(ORIGINAL_IMAGES_DIR, image_name))
        height, width = image.shape[:2]
        
        vein_instances = []
        class_names = []
        for label_name, label_polygons in labels.items():
            if label_name not in LABELBOX_CLASS_NAMES:
                continue

            class_names.append(label_name)
            for index, label_polygon in enumerate(label_polygons):
                polygon = []
                for coords in label_polygon:
                    polygon.append(
                        [coords['x'], height - coords['y']]
                    )
                bounding_box = BoundingBox(polygon)
                vein_instances.append({
                    'polygons': [polygon],
                    'bounding_box': [
                        bounding_box.minx, bounding_box.miny,
                        bounding_box.maxx, bounding_box.maxy
                    ],
                    "class": label_name
                })
        labelbox_images[image_name] = {
            'image': image,
            'id': item['ID'],
            'review_url': '{}/review/{}'.format(LABELBOX_PROJECT_URL, item['ID']),
            'vein_instances': vein_instances,
            'class_names': list(set(class_names))
        }
        
    return labelbox_images


In [ ]:
lb = parse_labelbox(LABELBOX_JSON_PATH)
predictions_json = json.load(open(PREDICTIONS_JSON_PATH))

# for prediction in predictions_json[:5]:
for prediction in np.random.choice(predictions_json, 50):
    image_name =  prediction["original_image_id"]
        
    display(HTML(
        '<a href="{}" target=_blank>{}</a> contains {}'.format(
            lb[image_name]['review_url'],
            image_name,
            ', '.join(lb[image_name]['class_names'])
        )
    ))

    
    display_instances('Without Label', lb[image_name]['image'], [], show_label=True)
    display_instances('Labelbox', lb[image_name]['image'], lb[image_name]['vein_instances'], show_label=True
    )
    
    prediction_image = imread(os.path.join(PADDED_IMAGES_DIR, image_name))
    display_instances('Prediction', prediction_image, prediction["vein_instances"], show_label=True)
